In [ ]:
#default_exp data.feature_calc

### Feature calculation
> This module calculates features/aggregations that could be useful to detect fraudulent transactions. Results are saved into 2 parquet datasets for features email and customer_id features. These features are calculated:

Email aggregations:
 * "num_customer_id_by_email": number of previously known different customer_ids for a given email, up to 10
 * "same_customer_id_by_email": 1 if order contains the same customer_id as the last order from this email
 * "known_customer_id_by_email": 1 if the customer_id has been seen before with the email
 * "order_amount_mean_by_email": mean of the last 10 order amounts on this email
 * "order_amount_std_by_email": std
 * "order_amount_min_by_email": min
 * "order_amount_max_by_email": max
 * "order_amount_sum_by_email": sum
  
Customer_id aggregations:
 * "num_email_by_customer_id": number of previosuly seen different emails on the order customer_id, up to 10
 * "num_ip_addr_by_customer_id": number of previously seen different IP addresses for the order customer_id, up to 10
 * "same_email_by_customer_id": 1 is the email is the same as latest order for this customer_id
 * "same_ip_addr_by_customer_id": 1 is the ip address is the same
 * "known_email_by_customer_id": 1 if the email was seen before with this customer_id
 * "known_ip_addr_by_customer_id": 1 if the IP address was seen befor with this customer_id
 * "order_amount_mean_by_customer_id": mean of the last 10 order amount on this customer_id
 * "order_amount_std_by_customer_id": std
 * "order_amount_min_by_customer_id": min
 * "order_amount_max_by_customer_id": max
 * "order_amount_sum_by_customer_id": sum

In [ ]:
#export
import dask.dataframe as dd
import numpy as np
import json

from hopeit.app.context import EventContext
from hopeit.app.events import Spawn, SHUFFLE
from hopeit.app.api import event_api
from hopeit.app.logger import app_logger

from fraud_poc.jobs import get_client, FeatureCalcJob, PreprocessingJob

In [ ]:
#export
__steps__ = ['run']

logger = app_logger()

In [ ]:
#export
def calculate(df, count_cols, stat_cols, by):
    counts = count_distinct_values(df, count_cols, by)
    stats = num_stats(df, stat_cols, by)
    right = counts.merge(stats)
    df = df.merge(right,
                  left_on=[df.index, 'order_id'], 
                  right_on=[by, 'order_id'],
                  suffixes=('', f'_DROP'))
    return df[[col for col in df.columns if col[-5:] != '_DROP']]
        

def count_distinct_values(df, cols, by):
    sets = []
    for col in cols:
        sets.append( 
            df.groupby([df.index, df.order_date, df.order_id])[col] \
                .apply(list) \
                .sort_index() \
                .groupby(level=0) \
                .apply(np.cumsum) \
                .apply(lambda x: list(x)[-10:]))
        
    counts = sets[0].to_frame()
    for col, result in zip(cols, sets):
        counts[col] = result
        counts[f'num_{col}_by_{by}'] = counts[col].apply(lambda x: len(set(x)))
        counts[f'last_{col}_by_{by}'] = counts[col].apply(lambda x: x[-2] if len(x)>1 else "")
        counts[f'same_{col}_by_{by}'] = counts[col].apply(lambda x: int(x[-2] == x[-1]) if len(x) > 1 else 0)
        counts[f'known_{col}_by_{by}'] = counts[col].apply(lambda x: int(x[-1] in x[:-1]) if len(x) > 1 else 0)
        counts[f'{col}_by_{by}'] = counts[col].apply(lambda x: json.dumps(x))
 
    count_cols = [f'{p}{col}_by_{by}' for p in ('', 'num_', 'last_', 'same_', 'known_') for col in cols]
    counts = counts.reset_index()[[by, 'order_id', *count_cols]]
    return counts

def num_stats(df, cols, by):
    results = []
    for col in cols:
        results.append(df.groupby([df.index, df.order_date, df.order_id])[col] \
                .apply(list) \
                .sort_index() \
                .groupby(level=0) \
                .apply(np.cumsum) \
                .apply(lambda x: list(x)[-10:]))
        
    stats = results[0].to_frame()
    for col, result in zip(cols[1:], results[1:]):
        stats[col] = result
    
    stats = stats.reset_index()[[by, 'order_id', *cols]]
    for col in cols:
        stats[f'{col}_mean_by_{by}'] = stats[col].apply(lambda x: np.mean(x))
        stats[f'{col}_std_by_{by}'] = stats[col].apply(lambda x: np.std(x))
        stats[f'{col}_min_by_{by}'] = stats[col].apply(lambda x: np.min(x))
        stats[f'{col}_max_by_{by}'] = stats[col].apply(lambda x: np.max(x))
        stats[f'{col}_sum_by_{by}'] = stats[col].apply(lambda x: np.sum(x))
        stats[f'{col}_by_{by}'] = stats[col].apply(lambda x: json.dumps(x))
    
    return stats


In [ ]:
#export
def run(job: PreprocessingJob, context: EventContext) -> FeatureCalcJob:
    base_path = context.env['data']['features']
    client = get_client(context)
    features = {}
    try:
        path = job.partitioned.get('customer_id')
        if path:
            logger.info(context, "Calculating features on customer_id...")
            df = dd.read_parquet(path, 
                         engine='fastparquet', 
                         columns=['order_id', 'order_date', 'email', 'ip_addr', 'order_amount'])
            df = df.map_partitions(calculate, count_cols=['email', 'ip_addr'], stat_cols=['order_amount'], by='customer_id')
            save_path = f'{base_path}/customer_id/'
            df.to_parquet(save_path)
            features['customer_id'] = save_path 
            logger.info(context, f"Saved {save_path}.")
        
        path = job.partitioned.get('email')
        if path:
            logger.info(context, "Calculating features on email...")
            df = dd.read_parquet(path, 
                         engine='fastparquet', 
                         columns=['order_id', 'order_date', 'customer_id', 'ip_addr', 'order_amount'])
            df = df.map_partitions(calculate, count_cols=['customer_id'], stat_cols=['order_amount'], by='email')
            save_path = f'{base_path}/email/'
            df.to_parquet(save_path)
            features['email'] = save_path 
            logger.info(context, f"Saved {save_path}.")
            
        return FeatureCalcJob(
            sources=job.partitioned,
            features=features
        )
    except Exception as e:
        logger.error(context, e)
        return None
    finally:
        client.close()

### Test from notebook

In [ ]:
from hopeit.testing.apps import config, execute_event

app_config = config('config/training-pipeline.json')
job = PreprocessingJob(source='./data/raw', partitioned={
    'customer_id': './data/partitioned/customer_id/', 
    'email': './data/partitioned/email'
})
result = await execute_event(app_config, 'data.feature-calc', job)
result

2020-07-08 11:12:58,114 | INFO | fraud-poc 0.0.1-training data.feature-calc leo-legion 35566 | Calculating features on customer_id... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-08T11:12:57.415253+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=
2020-07-08 11:15:10,771 | INFO | fraud-poc 0.0.1-training data.feature-calc leo-legion 35566 | Saved ./data/features/customer_id/. | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-08T11:12:57.415253+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=
2020-07-08 11:15:10,772 | INFO | fraud-poc 0.0.1-training data.feature-calc leo-legion 35566 | Calculating features on email... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-08T11:12:57.415253+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=
2020-07-08 11:16:38,234 | INFO | fraud-poc 0.0.1-training 

FeatureCalcJob(sources={'customer_id': './data/partitioned/customer_id/', 'email': './data/partitioned/email'}, features={'customer_id': './data/features/customer_id/', 'email': './data/features/email/'})

In [ ]:
df = dd.read_parquet(result.features['customer_id'])
df.head()

,order_id,order_date,email,ip_addr,order_amount,customer_id,email_by_customer_id,ip_addr_by_customer_id,num_email_by_customer_id,num_ip_addr_by_customer_id,...,same_email_by_customer_id,same_ip_addr_by_customer_id,known_email_by_customer_id,known_ip_addr_by_customer_id,order_amount_mean_by_customer_id,order_amount_std_by_customer_id,order_amount_min_by_customer_id,order_amount_max_by_customer_id,order_amount_sum_by_customer_id,order_amount_by_customer_id
0,43055b88-30c4-4224-8697-ab322ca0d642,2020-05-07 10:53:21+00:00,c981fe20491503912ff664740a3f97e2d77ddd6e,d957cc5adeeb4f39b29dd3cb84204f52177069e7,972.352792,00100242-b7fb-478f-ac50-16774cef232c,"[""c981fe20491503912ff664740a3f97e2d77ddd6e"", ""...","[""d957cc5adeeb4f39b29dd3cb84204f52177069e7"", ""...",1,2,...,1,1,1,1,522.676999,318.449988,89.088352,972.352792,5226.769995,"[567.6496593625884, 89.08835212177269, 216.277..."
1,3e40614a-cc08-4b2b-b974-765b1acfb9f7,2020-06-21 13:09:18+00:00,c981fe20491503912ff664740a3f97e2d77ddd6e,d957cc5adeeb4f39b29dd3cb84204f52177069e7,866.082201,00100242-b7fb-478f-ac50-16774cef232c,"[""c981fe20491503912ff664740a3f97e2d77ddd6e"", ""...","[""d957cc5adeeb4f39b29dd3cb84204f52177069e7"", ""...",1,1,...,1,1,1,1,613.614071,239.666862,310.594941,943.812111,6136.140712,"[943.8121110109099, 928.4246988157718, 310.594..."
2,52baa413-4853-45b2-8944-a32a3f1a7908,2020-04-07 08:51:12+00:00,c981fe20491503912ff664740a3f97e2d77ddd6e,d957cc5adeeb4f39b29dd3cb84204f52177069e7,233.483800,00100242-b7fb-478f-ac50-16774cef232c,"[""c981fe20491503912ff664740a3f97e2d77ddd6e"", ""...","[""d957cc5adeeb4f39b29dd3cb84204f52177069e7"", ""...",1,1,...,1,1,1,1,421.409455,210.804553,89.088352,834.000464,4214.094551,"[470.18093506588053, 573.7203277223783, 250.74..."
3,092cb774-2412-4bfb-bd61-0e55aebad1e0,2020-05-29 12:42:14+00:00,c981fe20491503912ff664740a3f97e2d77ddd6e,d957cc5adeeb4f39b29dd3cb84204f52177069e7,310.594941,00100242-b7fb-478f-ac50-16774cef232c,"[""c981fe20491503912ff664740a3f97e2d77ddd6e"", ""...","[""d957cc5adeeb4f39b29dd3cb84204f52177069e7"", ""...",1,1,...,1,1,1,1,707.163611,272.294281,135.586162,972.352792,7071.636113,"[748.7377739689492, 842.2944822793909, 972.352..."
4,a23c2b25-b815-47b9-9c27-195502c64e37,2019-08-22 11:21:32+00:00,c981fe20491503912ff664740a3f97e2d77ddd6e,dc7bb980ca2c5c80451196fb93f44f0e83e8f81f,598.571076,00100242-b7fb-478f-ac50-16774cef232c,"[""c981fe20491503912ff664740a3f97e2d77ddd6e"", ""...","[""d957cc5adeeb4f39b29dd3cb84204f52177069e7"", ""...",1,4,...,1,0,1,0,635.571967,275.372603,79.062306,959.817775,6355.719671,"[537.3812355224538, 334.91419993288116, 79.062..."


In [ ]:
dd.read_parquet(result.features['email']).head()

,order_id,order_date,customer_id,ip_addr,order_amount,email,customer_id_by_email,num_customer_id_by_email,last_customer_id_by_email,same_customer_id_by_email,known_customer_id_by_email,order_amount_mean_by_email,order_amount_std_by_email,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,order_amount_by_email
0,f0d63eec-8589-46fc-98aa-e9d49fead2eb,2020-05-27 17:28:52+00:00,bce8b787-e1b5-405c-9b3e-06fc91b31c36,7a4be0ba1a02ae4a27cf152848be77b35181b2ad,66.255347,00038ba252faef9a24b098ca6594c202b468e6db,"[""bce8b787-e1b5-405c-9b3e-06fc91b31c36"", ""bce8...",1,bce8b787-e1b5-405c-9b3e-06fc91b31c36,1,1,492.376981,321.362474,50.196317,974.606197,4923.769808,"[865.5761207952322, 529.3101147270868, 769.143..."
1,f8310a94-64aa-4938-89c8-574ca8d08e78,2020-06-30 22:54:51+00:00,bce8b787-e1b5-405c-9b3e-06fc91b31c36,dcf438634cfd4b0ad0d72cee70e638cea9fd9070,716.340468,00038ba252faef9a24b098ca6594c202b468e6db,"[""bce8b787-e1b5-405c-9b3e-06fc91b31c36"", ""bce8...",1,bce8b787-e1b5-405c-9b3e-06fc91b31c36,1,1,557.434306,251.412444,66.255347,913.562427,5574.343056,"[66.25534670339583, 245.04497173829088, 487.11..."
2,936cd7af-f2b7-4dd0-a10d-50f55951b7b5,2020-03-09 20:38:20+00:00,bce8b787-e1b5-405c-9b3e-06fc91b31c36,7a4be0ba1a02ae4a27cf152848be77b35181b2ad,912.128646,00038ba252faef9a24b098ca6594c202b468e6db,"[""bce8b787-e1b5-405c-9b3e-06fc91b31c36"", ""bce8...",1,bce8b787-e1b5-405c-9b3e-06fc91b31c36,1,1,482.109044,274.778605,72.114069,912.128646,4821.090443,"[555.6353073554191, 143.60326579705617, 72.114..."
3,07ed5409-b5ff-4a88-87b4-c6bd9ed29e1a,2020-04-11 09:36:36+00:00,bce8b787-e1b5-405c-9b3e-06fc91b31c36,7a4be0ba1a02ae4a27cf152848be77b35181b2ad,451.643311,00038ba252faef9a24b098ca6594c202b468e6db,"[""bce8b787-e1b5-405c-9b3e-06fc91b31c36"", ""bce8...",1,bce8b787-e1b5-405c-9b3e-06fc91b31c36,1,1,574.573381,237.089251,68.484679,912.128646,5745.733814,"[439.3317003697285, 544.3782438612366, 649.361..."
4,48fc743c-60af-42b0-8658-2fe82e6a8c08,2020-02-01 13:43:27+00:00,bce8b787-e1b5-405c-9b3e-06fc91b31c36,7a4be0ba1a02ae4a27cf152848be77b35181b2ad,72.114069,00038ba252faef9a24b098ca6594c202b468e6db,"[""bce8b787-e1b5-405c-9b3e-06fc91b31c36"", ""bce8...",1,bce8b787-e1b5-405c-9b3e-06fc91b31c36,1,1,368.225832,244.774665,72.114069,963.814082,3682.258322,"[392.77047614406723, 420.9638784860715, 254.77..."
